# Access to first Mosaic images of LSSTCam

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- member : DESC, rubin-inkind
- creation date : 2025-04-16
- last update : 2025-04-19

In [ ]:
import sys
import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
import numpy as np
import pandas as pd
from astropy.time import Time
#%matplotlib widget

In [ ]:
sys.path.append("../libs")

In [ ]:
from conversion import detector,rafts,ccds, dict_detector

In [ ]:
afwDisplay.setDefaultBackend("firefly")

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [ ]:
def displayExposure(exposure,title=None):
    afwDisplay.setDefaultBackend('matplotlib') 
    afwDisplay.setImageColormap(cmap='gray')
    fig = plt.figure(figsize=(10,10))
    afw_display = afwDisplay.Display(1)
    afw_display.scale('asinh', 'zscale')
    afw_display.mtv(exposure.getImage())
    plt.title(title)
    plt.gca().axis('off')
    return afw_display

def displayImage(image,title=None):
    afwDisplay.setDefaultBackend('matplotlib') 
    fig = plt.figure(figsize=(10,10))
    afw_display = afwDisplay.Display(1)
    afw_display.scale('asinh', 'zscale')
    #afw_display.scale('linear', min=-5, max=10)
    afw_display.setImageColormap(cmap='plasma')
    afw_display.mtv(image)
    plt.title(title)
    plt.gca().axis('off')
    return afw_display
    
def displayImageGhosts(image, zmin=0, zmax=5000, title=None):
    afwDisplay.setDefaultBackend('matplotlib') 
    fig = plt.figure(figsize=(10,10))
    afw_display = afwDisplay.Display(1)
    #afw_display.scale('asinh', 'zscale')
    afw_display.scale('linear', min=zmin, max=zmax)
    afw_display.setImageColormap(cmap='plasma')
    afw_display.mtv(image)
    plt.title(title)
    plt.gca().axis('off')
    return afw_display    

## RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

Existing collections:

    LSSTCam/runs/nightlyValidation/20250416/d_2025_04_15/DM-50157
    LSSTCam/runs/nightlyValidation/20250415/d_2025_04_15/DM-50157

## Configuration

### Butler and collection

In [ ]:
# Define butler
from lsst.daf.butler import Butler

repo = '/repo/embargo'
instrument = 'LSSTCam'
collection_validation  = instrument + '/runs/nightlyValidation'
collection_quicklook   = instrument + '/runs/quickLookTesting'
date_start = 20250415
date_selection = 20250416
where_clause = "instrument = \'" + f"{instrument}" +"\'"
where_clause_date = where_clause + f"and day_obs >= {date_start}"
skymap_name = "lsst_cells_v1"

In [ ]:
collection_validation = os.path.join(collection_validation,'20250416/d_2025_04_15/DM-50157') 

In [ ]:
butler = Butler(repo,collections=collection_validation)
registry = butler.registry

### Collections in the butler

In [ ]:
sorted(registry.queryCollections(expression = instrument+"/*"))

## Select the Instrument and observation date

In [ ]:
collection = collection_validation 
where_clause = "instrument = \'LSSTCam\' and day_obs >= 20250415"

In [ ]:
butler = Butler(repo,collections=collection)
registry = butler.registry

In [ ]:
print(butler.registry.dimensions["exposure"].RecordClass.fields)

## Dump registry into a pandas dataframe

In [ ]:
where_clause_date

In [ ]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','azimuth','zenith','science_program','jd','mjd'])
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= where_clause_date)):
    
    
    try:
        jd_start = info.timespan.begin.value
        jd_end = info.timespan.end.value
        the_Time_start = Time(jd_start,format="jd",scale="utc")
        the_Time_end = Time(jd_end,format="jd",scale="utc")
        mjd_start = the_Time_start.mjd
        mjd_end = the_Time_end.mjd
        isot_start = the_Time_start.isot
        isot_end = the_Time_end.isot
        
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,isot_start,isot_end ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                              jd_start, mjd_start]
    except:
        print(">>>   Unexpected error:", sys.exc_info()[0])
        info_timespan_begin_to_string = "2021-01-01 00:00:00.00"
        info_timespan_end_to_string = "2051-01-01 00:00:00.00"
        info_timespan_begin_jd = 0
        info_timespan_begin_mjd = 0
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,
                                  pd.to_datetime(info_timespan_begin_to_string),
                                  pd.to_datetime(info_timespan_end_to_string) ,
                                  info.observation_type, info.target_name, 
                                  info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info_timespan_begin_jd, info_timespan_begin_mjd  ]
 
    
    if count < 2:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        print(f"mjd = {mjd_start}, jd = {jd_start}")
    

In [ ]:
df_science = df_exposure[df_exposure.type == 'science']

In [ ]:
df_science

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,4))
df_science.expos.hist(ax=ax)

In [ ]:
#df_science = df_science[df_science.expos>=15]

In [ ]:
df_science.reset_index(drop=True,inplace=True)

In [ ]:
df_science

In [ ]:
id_min = df_science_id_min = df_science.id.min()
id_max = df_science_id_max = df_science.id.max()

In [ ]:
#! pip install --user openpyxl

In [ ]:
output_file_csv = f"lsstcam_lgb_{id_min}-{id_max}.csv"
output_file_xlsx = f"lsstcam_lgb_{id_min}-{id_max}.xlsx"
df_science.to_csv(output_file_csv)
#df_science.to_excel(output_file_xlsx)

## Dataset type

In [ ]:
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=collection).any(execute=False, exact=False):
        # Limit search results to the data products
        if ('_config' not in datasetType.name) and ('_log' not in datasetType.name) and ('_metadata' not in datasetType.name) and ('_resource_usage' not in datasetType.name):
            print(datasetType)

In [ ]:
dataProduct = 'preliminary_visit_image'
datasetRefs = list(butler.registry.queryDatasets(dataProduct,where=where_clause_date))

In [ ]:
len(datasetRefs)

In [ ]:
all_selected_visit = df_science.id.values
all_selected_visit = sorted(all_selected_visit)
all_selected_visit

> I was just poking around at images, and the following dataId has some really nice nebulosity:
> preliminary_visit_image@{instrument: 'LSSTCam', detector: 71, visit: 2025041500268, band: 'i', day_obs: 20250415, physical_filter: 'i_39'} [sc=ExposureF] (run=LSSTCam/runs/nightlyValidation/12 id=6695e83d-a251-440f-87c5-c85b5471b3ac)

In [ ]:
#the_selected_visit = all_selected_visit[3]
#the_selected_visit = 2025041500268
#the_selected_detector = 71
#title = f"dataProduct  v={the_selected_visit} d={the_selected_detector}"

In [ ]:
the_selected_visit = all_selected_visit[3]
the_selected_visit = 2025041700817
the_selected_detector = 71
title = f"dataProduct  v={the_selected_visit} d={the_selected_detector}"

## Select a Raft
![title](figs/LSSTCam_fp_layout.png)

In [ ]:
def getDetectorfromRaft(raft):
    all_det = []
    for ccd in ccds:
        _,det = detector(raft,ccd)
        all_det.append(det)
    return all_det

In [ ]:
print(rafts)

In [ ]:
print(ccds)

In [ ]:
the_raft_selected = "R22"
list_of_detectors = getDetectorfromRaft(the_raft_selected)
print(list_of_detectors)

## Mosaic

https://pipelines.lsst.io/py-api/lsst.afw.display.Mosaic.html#lsst.afw.display.Mosaic

In [ ]:
for i_raft in range(len(rafts)):
    for i_ccd in range(len(ccds)):
        detector_id=detector(rafts[i_raft],ccds[i_ccd])
        print('"'+str(detector_id[1]) + '": "' + rafts[i_raft]+ '_' + ccds[i_ccd]+'",',end="")
        #print(str(detector_id[1]) + ': ' + rafts[i_raft]+ '_' + ccds[i_ccd])

In [ ]:
list_of_detectors = getDetectorfromRaft(the_raft_selected)

In [ ]:
list_of_detectors

In [ ]:
detector_clause = f"detector in ("
for ii,det_id in enumerate(list_of_detectors):
    if ii<8:
        detector_clause += str(det_id) +","
    else:
         detector_clause += str(det_id) +")"

In [ ]:
detector_clause

In [ ]:
def queryExposures(expo="2025041500160",raft="R22"):

    list_of_detectors = getDetectorfromRaft(raft)
    detector_clause = f"detector in ("
    for ii,det_id in enumerate(list_of_detectors):
        if ii<len(list_of_detectors)-1:
            detector_clause += str(det_id) +","
        else:
             detector_clause += str(det_id) +")"
    where_clause = f"instrument='LSSTCam' AND visit={expo} AND "
    where_clause += detector_clause
    
    dataset_refs = butler.query_datasets("preliminary_visit_image", collections = collection,where = where_clause)
    exposures = [butler.get(dataset_ref) for dataset_ref in dataset_refs]
    return exposures

def make_mosaic(exposures, binning=4):
    from lsst.pipe.tasks.visualizeVisit import (
        VisualizeBinExpConfig,
        VisualizeBinExpTask,
        VisualizeMosaicExpConfig,
        VisualizeMosaicExpTask,
    )
    camera = butler.get("camera", collections=collection, instrument=instrument)
    
    visualizeBinExpConfig = VisualizeBinExpConfig()
    visualizeBinExpConfig.binning = binning
    visualizeBinExpTask = VisualizeBinExpTask(config=visualizeBinExpConfig)
    exposures_binned = [visualizeBinExpTask.run(inputExp = exposure, camera=camera).outputExp for exposure in exposures]
    
    visualizeMosaicExpConfig = VisualizeMosaicExpConfig()
    visualizeMosaicExpConfig.binning = binning
    visualizeMosaicExpTask = VisualizeMosaicExpTask(config=visualizeMosaicExpConfig)
    
    mosaic_full = visualizeMosaicExpTask.run(inputExps=exposures_binned, camera=camera)
    mosaic = mosaic_full.outputData
    return mosaic, mosaic_full

def show_ghosts(exp_id="2025041500160", raft="R22" ,binning=4, zmin=2230, zmax=2330):
    expos = queryExposures(exp_id,raft)
    mosaic, mosaic_full = make_mosaic(expos, binning)
    displayImageGhosts(mosaic, zmin=zmin, zmax=zmax)
    return mosaic, mosaic_full, expos

In [ ]:
the_list_of_exposures  = queryExposures(the_selected_visit, the_raft_selected)

In [ ]:
the_selected_visit

In [ ]:
mosaic, mosaic_full =  make_mosaic(the_list_of_exposures, binning=4)

In [ ]:
display = afwDisplay.Display(frame=1)
display.scale("asinh", "zscale")
display.mtv(mosaic, title= str(the_selected_visit) +'::'+the_raft_selected)

In [ ]:
#mosaic, mosaic_full, expos = show_ghosts(the_selected_visit, the_raft_selected ,binning=4, zmin=0, zmax=6000)

In [ ]:
#ghost_367, expos_367 = show_ghosts("2024112200367", binning=8, zmin=500, zmax=2000)

In [ ]:
#displayImageGhosts(ghost, zmin=820, zmax=900)

In [ ]:
#afwDisplay.setDefaultBackend('matplotlib') 
#fig = plt.figure(figsize=(10,10))
#afw_display = afwDisplay.Display(1)
#afw_display.scale('asinh', 'zscale')
#afw_display.scale('linear', min=zmin, max=zmax)
#afw_display.setImageColormap(cmap='grey')
#afw_display.mtv(ghost_367)
#plt.title("MC_2024112200367")
#plt.gca().axis('off')


In [ ]:
#ghost_292, expos_292 = show_ghosts("2024111100292", binning=8, zmin=500, zmax=2000)

In [ ]:
#afwDisplay.setDefaultBackend('matplotlib') 
#fig = plt.figure(figsize=(10,10))
#afw_display = afwDisplay.Display(1)
#afw_display.scale('asinh', 'zscale')
#afw_display.scale('linear', min=zmin, max=zmax)
#afw_display.setImageColormap(cmap='grey')
#afw_display.mtv(ghost_292)
#plt.title("MC_2024112200367")
#plt.gca().axis('off')

In [ ]:
type(mosaic)

In [ ]:
mosaic.getBBox()

In [ ]:
#expos_292[0].visitInfo.getBoresightRaDec()

In [ ]:
#expos_292[0].getMetadata()["RA"]

In [ ]:
#ghost_292.writeFits("ghost_292.fits", expos_292[0].getMetadata(), "w")

In [ ]:
#ghost_292.getDimensions()

In [ ]:
#ghost_367.writeFits("ghost_367.fits", expos_367[0].getMetadata(), "w")